In [1]:
import sys
sys.path = ["../../.."] + sys.path # 切换到项目目录下

import scanpy as sc
import scvelo as scv
import velovgi

from ray import tune, air
from ray.air import session

Global seed set to 0
/usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


## 1. 读取原始数据

In [2]:
adata = scv.read("../../../erythroid_lineage/data/erythroid_lineage.h5ad")

cluster_key = "celltype"
batch_key="stage"

adata

AnnData object with n_obs × n_vars = 500 × 53801
    obs: 'sample', 'stage', 'sequencing.batch', 'theiler', 'celltype'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'MURK_gene', 'Δm', 'scaled Δm'
    uns: 'celltype_colors'
    obsm: 'X_pca', 'X_umap'
    layers: 'spliced', 'unspliced'

In [3]:
batch_pair_list = [
    ["E7.0", "E7.25"],
    ["E7.25", "E7.5"],
    ["E7.5", "E7.75"],
    ["E7.75", "E8.0"],
    ["E8.0", "E8.25"],
    ["E8.25", "E8.5"],
]

In [4]:
cluster_edges = [
    ("Blood progenitors 1", "Blood progenitors 2"), 
    ("Blood progenitors 2", "Erythroid1"), 
    ("Erythroid1", "Erythroid2"), 
    ("Erythroid2", "Erythroid3")
    ] # 已知的细胞类型间的分化信息

## 2. 在预处理部分调整构建邻居数量的两个参数

1. 目标函数

In [5]:
from pytorch_lightning import loggers
from torch_geometric import seed_everything

def train_velovgi(config):

    # 邻居数量的两个参数
    n_bnn_neighbors = config["n_bnn_neighbors"]
    n_knn_neighbors = config["n_knn_neighbors"]
    
    knn_mask, bnn_mask, subsample_adata = velovgi.pp.preprocess(adata, n_bnn_neighbors, n_knn_neighbors, sample_mode="random", batch_key=batch_key, batch_pair_list=batch_pair_list)

    name = ""
    for k,v in config.items():
        name += "%s_%s,"%(k, v)
    name = name[:-1]

    seed_everything(0)
    # 模型训练
    logger = loggers.TensorBoardLogger(save_dir="./log", name=name)
    velovgi.tl.VELOVGI.setup_anndata(adata=subsample_adata, spliced_layer="Ms", unspliced_layer="Mu")
    velovgi_model = velovgi.tl.VELOVGI(subsample_adata)
    velovgi_model.train(logger=logger, max_epochs=50)

    # 模型恢复
    velovgi.tl.add_velovi_outputs_to_adata(subsample_adata, velovgi_model) # 模型输出
    velovgi.pp.moment_recover(adata, subsample_adata) # 恢复

    # 速率计算
    scv.tl.velocity_graph(adata)
    scv.pl.velocity_embedding(adata, color=cluster_key)
    scv.pl.velocity_embedding_stream(adata, color=cluster_key, save="%s.png"%name)

    # 计算指标评价
    adata_velo = velovgi.tl.pre_metric(adata)
    exp_metrics = velovgi.tl.summary_metric(adata_velo, cluster_edges, cluster_key)[-1] # 计算指标汇总后的结果

    session.report({"CBDir": exp_metrics["CBDir"], "ICVCoh": exp_metrics["ICVCoh"]})


2. 搜索空间，网格搜索

In [6]:
search_space = {
    "n_bnn_neighbors": tune.grid_search([15, 45]),
    "n_knn_neighbors": tune.grid_search([15),
}

3. 执行调参

In [7]:
from ray.tune.schedulers import ASHAScheduler

name = "test_experiment"

tuner = tune.Tuner(
    train_velovgi,
    tune_config=tune.TuneConfig(
        metric="CBDir",
        mode="max",
        scheduler=ASHAScheduler()
    ),
    run_config=air.RunConfig(
        local_dir="./results", # Trail内部具体输出结果在这里保存
        name=name # 开启调参的Tensorboard日志
    ),
    param_space=search_space,
)

results = tuner.fit()

2023-06-03 23:40:10,927	INFO worker.py:1625 -- Started a local Ray instance.
2023-06-03 23:40:12,744	INFO tune.py:218 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2023-06-03 23:40:13,086	INFO tensorboardx.py:172 -- pip install "ray[tune]" to see TensorBoard files.
2023-06-03 23:40:13,087	WARNING callback.py:142 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


2023-06-03 23:40:14,335	WARNING worker.py:1986 -- Warning: The actor ImplicitFunc is very large (26 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(pid=25135) Global seed set to 0
(pid=25135) /usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
(pid=25135)   new_rank_zero_deprecation(
(pid=25135) /usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zer

(train_velovgi pid=25135) Filtered out 51490 genes that are detected 20 counts (shared).
(train_velovgi pid=25135) Normalized count data: X, spliced, unspliced.
(train_velovgi pid=25135) Extracted 2000 highly variable genes.
(train_velovgi pid=25135) Logarithmized X.
(train_velovgi pid=25135) calculating knn and bnn mask...
(train_velovgi pid=25135) pair_list : [['E7.0', 'E7.25'], ['E7.25', 'E7.5'], ['E7.5', 'E7.75'], ['E7.75', 'E8.0'], ['E8.0', 'E8.25'], ['E8.25', 'E8.5']]


(pid=25199) Global seed set to 0
(pid=25199) /usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
(pid=25199)   new_rank_zero_deprecation(
(pid=25199) /usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
(pid=25199)   return new_rank_zero_deprecation(*args, **kwargs)


(train_velovgi pid=25199) Normalized count data: X, spliced, unspliced.
(train_velovgi pid=25199) Normalized count data: X, spliced, unspliced.
(train_velovgi pid=25199) Extracted 2000 highly variable genes.
(train_velovgi pid=25199) Logarithmized X.
(train_velovgi pid=25199) calculating knn and bnn mask...
(train_velovgi pid=25199) pair_list : [['E7.0', 'E7.25'], ['E7.25', 'E7.5'], ['E7.5', 'E7.75'], ['E7.75', 'E8.0'], ['E8.0', 'E8.25'], ['E8.25', 'E8.5']]


...

2023-06-03 23:41:03,489	ERROR ray_trial_executor.py:883 -- An exception occurred when trying to stop the Ray actor:Traceback (most recent call last):
  File "/usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/ray/tune/execution/ray_trial_executor.py", line 874, in _resolve_stop_event
    ray.get(future, timeout=timeout)
  File "/usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/ray/_private/worker.py", line 2523, in get
    raise value
ray.exceptions.OutOfMemoryError: Task was killed due to the node running low on memory.
Memory on the node (IP: 172.24.199.116, ID: 2dfdba42a28fe9d17bbfb37d7aba897717420052661643c79e1b9214) where the task (actor ID: 9d1379104de1f40573f83dcc01000000, name=ImplicitFunc.__init__, pid=25350, memory used=0.51GB) was running was 11.79GB / 12.39GB (0.951635), which exceeds the memory us

Epoch 4/50:   6%|▌         | 3/50 [00:11<02:54,  3.71s/it, loss=1.87e+06, v_num=0] [repeated 3x across cluster]


(raylet) [2023-06-03 23:41:10,847 E 24460 24460] (raylet) node_manager.cc:3071: 5 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 2dfdba42a28fe9d17bbfb37d7aba897717420052661643c79e1b9214, IP: 172.24.199.116) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.24.199.116`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.


Epoch 50/50: 100%|██████████| 50/50 [02:05<00:00,  2.52s/it, loss=1.54e+06, v_num=0]


(train_velovgi pid=25135) `Trainer.fit` stopped: `max_epochs=50` reached.


4. 查看训练日志与最优结果: tensorboard --logdir

In [ ]:
results.get_best_result().config

2023-06-03 21:51:18,641	WARNING experiment_analysis.py:644 -- Could not find best trial. Did you pass the correct `metric` parameter?


RuntimeError: No best trial found for the given metric: CBDir. This means that no trial has reported this metric, or all values reported for this metric are NaN. To not ignore NaN values, you can set the `filter_nan_and_inf` arg to False.